## PyCity Schools Analysis
### Summary of the analysis
This analysis looks at how many schools are in the district, how many students there are, the total budget, the average math and reading scores of students in the district and the percentage of students that passed (passing grade is 70%). The analysis also summarizes this data at a school-specific level, grade-specific level, and identifies the top performing schools and lowest performing schools based on the number of students that passed both math and reading compared against the total number of students at each school.

The analysis takes a deeper dive into the data to determine if there are any correlations between the number of students passing math & reading and the school's spending, size, and type.

### Draw two conclusions or comparisons from the calculations
Conclusion 1: Having a higher budget per student does not guarantee that there is a higher probability of students succeeding and passing math & reading. After analyzing the overall passing percentage by spendings per student, the summary shows that the lowest spending bin (<$585 per student) has the highest percentage of students passing.

Conclusion 2: Students that attend Charter schools have higher average math and reading scores, a higher passing rate in both math and reading, and a higher overall passing rate compared to students at District schools.


In [1]:
import pandas as pd
from pathlib import Path

In [2]:
school_data_path = Path("Resources/schools_complete.csv")
student_data_path = Path("Resources/students_complete.csv")
school_data = pd.read_csv(school_data_path)
student_data = pd.read_csv(student_data_path)

school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head().style.hide(axis='index')

Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

In [3]:
#Count distinct school names
distinct_school_count = school_data.school_name.nunique()
distinct_school_count

15

In [4]:
#Count distinct student IDs
#Rename Student ID column to student_id
student_id_df = student_data.rename(columns={"Student ID":"student_id"})
#Get distinct count of student_id values
distinct_student_id_count = student_id_df.student_id.nunique()
distinct_student_id_count

39170

In [5]:
#Calculate total budget
total_budget = school_data['budget'].sum()
total_budget

24649428

In [6]:
#Calculate average math score
average_math_score = student_data['math_score'].mean()
average_math_score

78.98537145774827

In [7]:
#Calculate average reading score
average_reading_score = student_data['reading_score'].mean()
average_reading_score

81.87784018381414

In [8]:
#Calculate percentage of students who passed math
passing_math_count = school_data_complete[(school_data_complete["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = passing_math_count / float(distinct_student_id_count) * 100
passing_math_percentage

74.9808526933878

In [9]:
#Calculate percentage of students who passed reading
passing_reading_count = school_data_complete[(school_data_complete["reading_score"] >= 70)].count()["student_name"]
passing_reading_percentage = passing_reading_count / float(distinct_student_id_count) * 100
passing_reading_percentage

85.80546336482001

In [10]:
#Calculate percentage of students who passed math and reading
passing_math_reading_count = school_data_complete[
    (school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)
].count()["student_name"]
overall_passing_rate = passing_math_reading_count /  float(distinct_student_id_count) * 100
overall_passing_rate

65.17232575950983

In [11]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
district_summary = pd.DataFrame([{
    'Total Schools': distinct_school_count,
    'Total Students': distinct_student_id_count,
    'Total Budget': total_budget,
    'Average Math Score': average_math_score,
    'Average Reading Score': average_reading_score,
    '% Passing Math': passing_math_percentage,
    '% Passing Reading': passing_reading_percentage,
    '% Overall Passing': overall_passing_rate
}])

# Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

# Display the DataFrame
district_summary = district_summary.style.hide(axis='index')
district_summary

Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
15,"39,170","$24,649,428.00",78.985371,81.877840,74.980853,85.805463,65.172326


## School Summary

In [12]:
#Get list of distinct school names
student_count = school_data_complete.groupby(['school_name']).size().reset_index(name='count')
merged_data = pd.merge(student_count, school_data, how="left", on=["school_name", "school_name"])
school_names = merged_data['school_name']
#school_names

In [13]:
#Get list of school types
school_types = merged_data['type']
#school_types

In [14]:
#Get count of students per school
per_school_counts = merged_data['count']
#per_school_counts

In [15]:
#Get budget per school
per_school_budget = merged_data['budget']
#per_school_budget

In [16]:
#Calculate budget per student per school
per_school_capita = per_school_budget / per_school_counts
#per_school_capita

In [17]:
# Calculate the average math scores per school
#Group data based on school name
grouped_data = school_data_complete.groupby(['school_name'])
school_math = grouped_data['math_score'].mean().reset_index()
per_school_math = school_math['math_score']
#per_school_math

In [18]:
# Calculate the average reading scores per school
school_reading = grouped_data['reading_score'].mean().reset_index()
per_school_reading = school_reading['reading_score']
#per_school_reading

In [19]:
# Use the provided code to calculate the number of students per school that passed both math and reading with scores of 70 or higher
students_passing_math = school_data_complete[
    (school_data_complete["math_score"] >= 70)
]
school_students_passing_math = students_passing_math.groupby(["school_name"]).size().reset_index(name='math pass count')
school_students_passing_math = school_students_passing_math['math pass count']
school_students_passing_math = (school_students_passing_math / per_school_counts) * 100
#school_students_passing_math

In [20]:
# Use the provided code to calculate the number of students per school that passed both math and reading with scores of 70 or higher
students_passing_reading = school_data_complete[
    (school_data_complete["reading_score"] >= 70)
]
school_students_passing_reading = students_passing_reading.groupby(["school_name"]).size().reset_index(name='reading pass count')
school_students_passing_reading = school_students_passing_reading['reading pass count']
school_students_passing_reading = (school_students_passing_reading / per_school_counts) * 100
#school_students_passing_reading

In [21]:
# Use the provided code to calculate the number of students per school that passed both math and reading with scores of 70 or higher
students_passing_math_and_reading = school_data_complete[
    (school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)
]
school_students_passing_math_and_reading = students_passing_math_and_reading.groupby(["school_name"]).size().reset_index(name='math reading pass count')
school_students_passing_math_and_reading = school_students_passing_math_and_reading['math reading pass count']
school_students_passing_math_and_reading = (school_students_passing_math_and_reading / per_school_counts) * 100
#school_students_passing_math_and_reading

In [22]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.
per_school_summary = pd.DataFrame({
    'School Name': school_names,
    'School Types': school_types,
    'Student Count': per_school_counts,
    'Total School Budget': per_school_budget,
    'Per Student Budget': per_school_capita,
    'Average Math Score': per_school_math,
    'Average Reading Score': per_school_reading,
    '% Passing Math': school_students_passing_math,
    '% Passing Reading': school_students_passing_reading,
    '% Overall Passing': school_students_passing_math_and_reading
})
# Formatting
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)

# Display the DataFrame
per_school_summary.style.hide(axis='index')

School Name,School Types,Student Count,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


 ## Highest-Performing Schools (by % Overall Passing)

In [23]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
top_schools = per_school_summary.sort_values(by='% Overall Passing', ascending=False).reset_index(drop="True")
top_schools.head(5).style.hide(axis='index')

School Name,School Types,Student Count,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

In [24]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.
bottom_schools = per_school_summary.sort_values(by='% Overall Passing', ascending=True).reset_index(drop="True")
bottom_schools.head(5).style.hide(axis='index')

School Name,School Types,Student Count,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Math scores by grade

In [25]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by `school_name` and take the mean of the the `reading_score` column for each.
ninth_grade_math_scores = ninth_graders.groupby(['school_name'])['math_score'].mean()
#ninth_grade_reading_scores
tenth_grader_math_scores = tenth_graders.groupby(['school_name'])['math_score'].mean()
eleventh_grader_math_scores = eleventh_graders.groupby(['school_name'])['math_score'].mean()
twelfth_grader_math_scores = twelfth_graders.groupby(['school_name'])['math_score'].mean()

# Combine each of the scores above into single DataFrame called `reading_scores_by_grade`
math_scores_by_grade = pd.DataFrame({
    '9th': ninth_grade_math_scores,
    '10th': tenth_grader_math_scores,
    '11th': eleventh_grader_math_scores,
    '12th': twelfth_grader_math_scores
})
# Minor data wrangling
math_scores_by_grade = math_scores_by_grade[["9th", "10th", "11th", "12th"]]
math_scores_by_grade.index.name = None

# Display the DataFrame
math_scores_by_grade

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading score by grade

In [26]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by `school_name` and take the mean of the the `reading_score` column for each.
ninth_grade_reading_scores = ninth_graders.groupby(['school_name'])['reading_score'].mean()
#ninth_grade_reading_scores
tenth_grader_reading_scores = tenth_graders.groupby(['school_name'])['reading_score'].mean()
eleventh_grader_reading_scores = eleventh_graders.groupby(['school_name'])['reading_score'].mean()
twelfth_grader_reading_scores = twelfth_graders.groupby(['school_name'])['reading_score'].mean()

# Combine each of the scores above into single DataFrame called `reading_scores_by_grade`
reading_scores_by_grade = pd.DataFrame({
    '9th': ninth_grade_reading_scores,
    '10th': tenth_grader_reading_scores,
    '11th': eleventh_grader_reading_scores,
    '12th': twelfth_grader_reading_scores
})
# Minor data wrangling
reading_scores_by_grade = reading_scores_by_grade[["9th", "10th", "11th", "12th"]]
reading_scores_by_grade.index.name = None

# Display the DataFrame
reading_scores_by_grade

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by school spending

In [27]:
# Establish the bins 
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [28]:
# Create a copy of the school summary since it has the "Per Student Budget" 
school_spending_df = per_school_summary.copy()
#school_spending_df.style.hide(axis='index')

In [29]:
# Use `pd.cut` to categorize spending based on the bins.
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(per_school_capita,spending_bins, labels=labels, include_lowest=True)
school_spending_df.style.hide(axis='index')

School Name,School Types,Student Count,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Spending Ranges (Per Student)
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283,$585-630
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769,<$585
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476,$630-645
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887,$630-645
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455,$585-630
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508,$645-680
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166,<$585
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884,$645-680
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172,$645-680
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541,$585-630


In [30]:
#  Calculate averages for the desired columns. 
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Math Score"].mean()
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Math"].mean()
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean()
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Overall Passing"].mean()

In [31]:
# Assemble into DataFrame
spending_summary = pd.DataFrame({
    'Average Math Score': spending_math_scores,
    'Average Reading Score': spending_reading_scores,
    '% Passing Math': spending_passing_math,
    '% Passing Reading': spending_passing_reading,
    '% Overall Passing': overall_passing_spending
})

# Display results
spending_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,90.369459
$585-630,81.899826,83.155286,87.133538,92.718205,81.418596
$630-645,78.518855,81.624473,73.484209,84.391793,62.857656
$645-680,76.997210,81.027843,66.164813,81.133951,53.526855


## Scores by school size

In [32]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [33]:
school_size_df = per_school_summary.copy()
#school_size_df.style.hide(axis='index')

In [34]:
# Categorize the spending based on the bins
# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.

school_size_df["School Size"] = pd.cut(per_school_counts, size_bins, labels=labels, include_lowest=True)
school_size_df.style.hide(axis='index')

School Name,School Types,Student Count,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,School Size
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283,Large (2000-5000)
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769,Medium (1000-2000)
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476,Large (2000-5000)
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887,Large (2000-5000)
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455,Medium (1000-2000)
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508,Large (2000-5000)
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166,Small (<1000)
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884,Large (2000-5000)
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172,Large (2000-5000)
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541,Small (<1000)


In [35]:
# Calculate averages for the desired columns. 
size_math_scores = school_size_df.groupby(["School Size"])["Average Math Score"].mean()
size_reading_scores = school_size_df.groupby(["School Size"])["Average Reading Score"].mean()
size_passing_math = school_size_df.groupby(["School Size"])["% Passing Math"].mean()
size_passing_reading = school_size_df.groupby(["School Size"])["% Passing Reading"].mean()
size_overall_passing = school_size_df.groupby(["School Size"])["% Overall Passing"].mean()

In [36]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`
size_summary = pd.DataFrame({
    'Average Math Score': size_math_scores,
    'Average Reading Score': size_reading_scores,
    '% Passing Math': size_passing_math,
    '% Passing Reading': size_passing_reading,
    '% Overall Passing': size_overall_passing
})

# Display results
size_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


## Scores by School Type

In [37]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
average_math_score_by_type = per_school_summary.groupby(["School Types"])["Average Math Score"].mean()
average_reading_score_by_type = per_school_summary.groupby(["School Types"])["Average Reading Score"].mean()
average_percent_passing_math_by_type = per_school_summary.groupby(["School Types"])["% Passing Math"].mean()
average_percent_passing_reading_by_type = per_school_summary.groupby(["School Types"])["% Passing Reading"].mean()
average_percent_overall_passing_by_type = per_school_summary.groupby(["School Types"])["% Overall Passing"].mean()

In [38]:
# Assemble the new data by type into a DataFrame called `type_summary`
type_summary = pd.DataFrame({
    'Average Math Score': average_math_score_by_type,
    'Average Reading Score': average_reading_score_by_type,
    '% Passing Math': average_percent_passing_math_by_type,
    '% Passing Reading': average_percent_passing_reading_by_type,
    '% Overall Passing': average_percent_overall_passing_by_type
})

# Display results
type_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Types,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
